In [18]:
import os.path
import re
import tarfile
import io
import smart_open
import pandas as pd

In [6]:

def extract_documents():
    fname = r'C:\Users\anupa\Downloads\nips12raw_str602.tgz'

    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')

docs = list(extract_documents())

In [7]:
print(len(docs))
print(docs[0][:500])

1740
1 
CONNECTIVITY VERSUS ENTROPY 
Yaser S. Abu-Mostafa 
California Institute of Technology 
Pasadena, CA 91125 
ABSTRACT 
How does the connectivity of a neural network (number of synapses per 
neuron) relate to the complexity of the problems it can handle (measured by 
the entropy)? Switching theory would suggest no relation at all, since all Boolean 
functions can be implemented using a circuit with very low connectivity (e.g., 
using two-input NAND gates). However, for a network that learns a pr


In [8]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [9]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [10]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [11]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [12]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [13]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8644
Number of documents: 1740


In [14]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [15]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.1459.
[([(0.02214648, 'neuron'),
   (0.012219782, 'cell'),
   (0.008834716, 'circuit'),
   (0.007435569, 'spike'),
   (0.006887154, 'synaptic'),
   (0.006712112, 'signal'),
   (0.006093828, 'chip'),
   (0.0058580097, 'analog'),
   (0.0054700268, 'firing'),
   (0.0053621805, 'response'),
   (0.005056121, 'voltage'),
   (0.0046402416, 'frequency'),
   (0.0041950713, 'activity'),
   (0.004020153, 'fig'),
   (0.0039565535, 'potential'),
   (0.00388635, 'noise'),
   (0.0038731075, 'memory'),
   (0.003832428, 'synapsis'),
   (0.003729047, 'connection'),
   (0.0036637208, 'threshold')],
  -0.9366091487241189),
 ([(0.007785022, 'hidden'),
   (0.0070547895, 'generalization'),
   (0.005762403, 'training_set'),
   (0.0056044147, 'layer'),
   (0.0053311307, 'prediction'),
   (0.005058552, 'hidden_unit'),
   (0.004456102, 'trained'),
   (0.0041153934, 'distance'),
   (0.0039601927, 'table'),
   (0.003952076, 'classification'),
   (0.0037964392, 'net'),
   (0.003092114, '

In [23]:
import pyLDAvis.gensim
 
lda_display = pyLDAvis.gensim.prepare(model, corpus, 
                         dictionary, sort_topics=False)

C:\Users\anupa\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [27]:
pyLDAvis.display(lda_display)

In [16]:
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score']
    return topic_details_df

In [20]:
contents = pd.DataFrame({'Original text':docs})
topic_details = pd.concat([get_topic_details(model,
                           corpus), contents], axis=1)
topic_details.head()

,Dominant_Topic,% Score,Original text
0,2.0,0.690645,"[connectivity, versus, entropy, yaser, abu, mo..."
1,5.0,0.534330,"[stochastic, learning, network, and, their, el..."
2,3.0,0.764972,"[learning, on, general, network, amir, atiya, ..."
3,5.0,0.369315,"[an, artificial, neural, network, for, spatio,..."
4,2.0,0.390147,"[on, property, of, network, of, neuron, like, ..."


In [33]:
model.get_document_topics(corpus, minimum_probability=0.0)


In [35]:

for doc in docs:
    print(model.get_document_topics(dictionary.doc2bow(doc)))

[(0, 4.7439316e-05), (1, 6.2762345e-05), (2, 0.69064337), (3, 9.184387e-05), (4, 5.0982057e-05), (5, 0.20695902), (6, 4.6833586e-05), (7, 0.0897487), (8, 0.01228629), (9, 6.272761e-05)]
[(0, 2.044283e-05), (1, 0.19670828), (2, 2.8832805e-05), (3, 0.12803565), (4, 2.1969488e-05), (5, 0.5343297), (6, 2.0181804e-05), (7, 0.06882869), (8, 0.07197925), (9, 2.7030952e-05)]
[(0, 0.056636192), (1, 5.039027e-05), (2, 5.371948e-05), (3, 0.76497287), (4, 4.093218e-05), (5, 0.040685967), (6, 3.7601476e-05), (7, 2.9114803e-05), (8, 0.13744287), (9, 5.036238e-05)]
[(0, 2.7466209e-05), (1, 3.633787e-05), (2, 3.873866e-05), (3, 0.25523636), (4, 0.17600279), (5, 0.3693149), (6, 2.7115504e-05), (7, 2.0995521e-05), (8, 0.19925895), (9, 3.631776e-05)]
[(0, 1.9560723e-05), (1, 2.587889e-05), (2, 0.39015725), (3, 0.23982398), (4, 2.1021506e-05), (5, 0.27885363), (6, 1.9310959e-05), (7, 1.4952466e-05), (8, 0.091038555), (9, 2.5864567e-05)]
[(0, 0.0056679747), (1, 0.34404248), (2, 0.13997357), (3, 0.17033887)

[(0, 0.6237831), (1, 2.4273057e-05), (2, 2.5876743e-05), (3, 3.5520276e-05), (4, 1.9717083e-05), (5, 2.6300195e-05), (6, 1.811268e-05), (7, 0.036177073), (8, 0.15712151), (9, 0.18276851)]
[(0, 3.145276e-05), (1, 4.1612086e-05), (2, 0.103124164), (3, 0.85139406), (4, 3.3801633e-05), (5, 0.045248467), (6, 3.1051153e-05), (7, 2.404289e-05), (8, 2.9752047e-05), (9, 4.1589057e-05)]
[(0, 2.9658242e-05), (1, 3.9237937e-05), (2, 4.1830328e-05), (3, 5.7419235e-05), (4, 3.1873104e-05), (5, 0.99968076), (6, 2.9279548e-05), (7, 2.267114e-05), (8, 2.8054565e-05), (9, 3.9216222e-05)]
[(0, 2.2822956e-05), (1, 0.059558026), (2, 3.218976e-05), (3, 4.4185963e-05), (4, 2.452736e-05), (5, 0.77881765), (6, 2.2531538e-05), (7, 1.7446158e-05), (8, 0.1614304), (9, 3.0178122e-05)]
[(0, 4.299155e-05), (1, 5.687794e-05), (2, 6.0635783e-05), (3, 0.5244468), (4, 4.6202134e-05), (5, 0.39281002), (6, 4.244261e-05), (7, 3.286329e-05), (8, 0.054139398), (9, 0.028321808)]
[(0, 2.4426978e-05), (1, 3.231696e-05), (2, 0.0

[(0, 4.497232e-05), (1, 0.17989682), (2, 6.342948e-05), (3, 0.5500802), (4, 0.17711483), (5, 6.446745e-05), (6, 4.4398086e-05), (7, 3.4377415e-05), (8, 0.09259704), (9, 5.9465572e-05)]
[(0, 0.0876426), (1, 0.53933), (2, 4.30756e-05), (3, 0.31057304), (4, 3.282195e-05), (5, 4.3780496e-05), (6, 3.0151188e-05), (7, 0.045042045), (8, 2.8889735e-05), (9, 0.017233536)]
[(0, 0.053340178), (1, 0.09067286), (2, 3.9641647e-05), (3, 0.45081455), (4, 3.020541e-05), (5, 4.029035e-05), (6, 2.774756e-05), (7, 2.148492e-05), (8, 0.40497592), (9, 3.7164315e-05)]
[(0, 4.0017818e-05), (1, 0.79923177), (2, 0.13390513), (3, 7.7475626e-05), (4, 4.3006326e-05), (5, 5.736522e-05), (6, 3.950685e-05), (7, 3.059013e-05), (8, 0.06652227), (9, 5.2914384e-05)]
[(0, 3.855327e-05), (1, 0.16536155), (2, 5.437598e-05), (3, 0.41964197), (4, 4.1432406e-05), (5, 0.31189433), (6, 3.8061e-05), (7, 2.9470613e-05), (8, 0.10284926), (9, 5.0977855e-05)]
[(0, 3.963983e-05), (1, 0.6374297), (2, 5.5908476e-05), (3, 0.2778277), (4,

[(0, 0.04424965), (1, 6.4963606e-05), (2, 0.043224923), (3, 0.082166076), (4, 5.277015e-05), (5, 0.51129955), (6, 4.8476177e-05), (7, 3.753508e-05), (8, 4.644805e-05), (9, 0.31880966)]
[(0, 3.4825873e-05), (1, 4.607473e-05), (2, 4.9118815e-05), (3, 0.3718818), (4, 0.09319331), (5, 4.9922608e-05), (6, 0.16246636), (7, 2.6621343e-05), (8, 0.10006364), (9, 0.27218837)]
[(0, 3.731705e-05), (1, 4.937056e-05), (2, 5.26324e-05), (3, 0.3262254), (4, 4.0103867e-05), (5, 0.027845263), (6, 3.6840564e-05), (7, 2.852563e-05), (8, 0.4731603), (9, 0.17252424)]
[(0, 4.891852e-05), (1, 6.471934e-05), (2, 0.0657901), (3, 0.71079814), (4, 5.2571726e-05), (5, 0.094679266), (6, 4.82939e-05), (7, 3.739394e-05), (8, 0.1284159), (9, 6.468352e-05)]
[(0, 3.0361985e-05), (1, 4.016899e-05), (2, 4.2822896e-05), (3, 5.8781552e-05), (4, 3.2629403e-05), (5, 4.3523658e-05), (6, 2.9974306e-05), (7, 2.320909e-05), (8, 0.99965835), (9, 4.014676e-05)]
[(0, 0.045923635), (1, 4.0808223e-05), (2, 0.008565818), (3, 0.05852108

[(0, 0.24871352), (1, 0.37755045), (2, 0.28050146), (3, 0.034417648), (4, 3.5976384e-05), (5, 0.012896796), (6, 3.304894e-05), (7, 2.5589776e-05), (8, 0.045781195), (9, 4.4264838e-05)]
[(0, 0.68750036), (1, 5.459269e-05), (2, 5.819955e-05), (3, 7.9888865e-05), (4, 4.434582e-05), (5, 5.9151935e-05), (6, 4.0737345e-05), (7, 0.31206912), (8, 3.903299e-05), (9, 5.4562475e-05)]
[(0, 0.14658336), (1, 0.15520011), (2, 7.7446675e-05), (3, 0.00010630881), (4, 5.901139e-05), (5, 0.39858106), (6, 5.420956e-05), (7, 4.1974432e-05), (8, 5.1941563e-05), (9, 0.2992446)]
[(0, 0.21038854), (1, 0.67440313), (2, 5.1165996e-05), (3, 7.0233924e-05), (4, 3.8986524e-05), (5, 0.10019631), (6, 3.581414e-05), (7, 2.773087e-05), (8, 3.4315766e-05), (9, 0.014753789)]
[(0, 0.023215592), (1, 0.072271064), (2, 5.937062e-05), (3, 0.74190986), (4, 4.5238132e-05), (5, 6.034218e-05), (6, 4.155705e-05), (7, 3.2177602e-05), (8, 0.16230917), (9, 5.5660366e-05)]
[(0, 4.594906e-05), (1, 6.079073e-05), (2, 6.4807085e-05), (3,

...

[(0, 3.0890573e-05), (1, 4.086831e-05), (2, 0.17074715), (3, 5.9805025e-05), (4, 3.3197462e-05), (5, 4.428138e-05), (6, 0.5224015), (7, 2.3613147e-05), (8, 2.9220259e-05), (9, 0.3065895)]
[(0, 3.5114714e-05), (1, 4.6456862e-05), (2, 0.14303006), (3, 0.19121438), (4, 3.773706e-05), (5, 5.0336654e-05), (6, 0.6654794), (7, 2.6842134e-05), (8, 3.321599e-05), (9, 4.643115e-05)]
[(0, 3.270577e-05), (1, 4.326983e-05), (2, 0.29162478), (3, 0.034144685), (4, 3.514822e-05), (5, 4.6883455e-05), (6, 0.124253355), (7, 2.500071e-05), (8, 3.093731e-05), (9, 0.54976326)]
[(0, 3.6529167e-05), (1, 4.832819e-05), (2, 0.5746242), (3, 0.25107646), (4, 3.9257146e-05), (5, 5.2364263e-05), (6, 3.6062742e-05), (7, 2.7923363e-05), (8, 3.4553967e-05), (9, 0.17402436)]
[(0, 3.1702428e-05), (1, 4.19424e-05), (2, 4.4713473e-05), (3, 6.137671e-05), (4, 3.4069948e-05), (5, 4.5445173e-05), (6, 3.1297634e-05), (7, 2.4233741e-05), (8, 2.9988218e-05), (9, 0.9996552)]
[(0, 3.3034743e-05), (1, 4.3705055e-05), (2, 0.1000032

[(0, 3.800824e-05), (1, 5.0285005e-05), (2, 0.03613549), (3, 0.9635281), (4, 4.0846673e-05), (5, 5.44845e-05), (6, 3.752293e-05), (7, 2.9053983e-05), (8, 3.595306e-05), (9, 5.0257175e-05)]
[(0, 3.0272462e-05), (1, 4.0050552e-05), (2, 0.16503236), (3, 0.12342158), (4, 3.2533193e-05), (5, 4.339533e-05), (6, 0.71130806), (7, 2.3140658e-05), (8, 2.8635572e-05), (9, 4.0028386e-05)]
[(0, 0.007478762), (1, 0.32284626), (2, 0.66938555), (3, 6.619452e-05), (4, 3.674429e-05), (5, 4.9012422e-05), (6, 3.3754364e-05), (7, 2.6135986e-05), (8, 3.2342163e-05), (9, 4.5209665e-05)]
[(0, 3.2892116e-05), (1, 0.33432034), (2, 0.61133194), (3, 0.038037706), (4, 3.534848e-05), (5, 4.715058e-05), (6, 3.247213e-05), (7, 2.5143154e-05), (8, 0.016093563), (9, 4.3492277e-05)]
[(0, 3.168265e-05), (1, 4.191624e-05), (2, 0.14712295), (3, 0.4261243), (4, 3.4048695e-05), (5, 4.5416822e-05), (6, 0.4265036), (7, 2.4218625e-05), (8, 2.9969511e-05), (9, 4.189304e-05)]
[(0, 0.08748128), (1, 5.35573e-05), (2, 0.16628172), (

[(0, 2.922928e-05), (1, 0.34468046), (2, 0.4498858), (3, 5.6588662e-05), (4, 3.1412106e-05), (5, 4.1899933e-05), (6, 0.087809354), (7, 2.2343236e-05), (8, 2.7648797e-05), (9, 0.117415234)]
[(0, 0.05703321), (1, 0.16909675), (2, 3.707021e-05), (3, 5.088506e-05), (4, 0.547761), (5, 3.7676837e-05), (6, 0.011444216), (7, 2.0091258e-05), (8, 2.486207e-05), (9, 0.21449421)]
[(0, 2.7683303e-05), (1, 3.662509e-05), (2, 0.49553284), (3, 5.359549e-05), (4, 0.1587247), (5, 3.968379e-05), (6, 0.20497186), (7, 2.1161472e-05), (8, 2.6186415e-05), (9, 0.14056565)]
[(0, 3.4473665e-05), (1, 4.560876e-05), (2, 0.025922094), (3, 0.7910443), (4, 3.704814e-05), (5, 4.941772e-05), (6, 0.07187295), (7, 2.635211e-05), (8, 3.260961e-05), (9, 0.11093517)]
[(0, 0.094115615), (1, 5.158425e-05), (2, 0.17579997), (3, 0.10087692), (4, 4.190205e-05), (5, 0.07139804), (6, 3.849243e-05), (7, 2.9804669e-05), (8, 0.029211517), (9, 0.5284361)]
[(0, 0.21705666), (1, 0.15931715), (2, 0.2314425), (3, 5.363175e-05), (4, 0.253

[(0, 2.527246e-05), (1, 3.3435535e-05), (2, 3.564457e-05), (3, 0.84735507), (4, 2.7159793e-05), (5, 3.6227866e-05), (6, 0.121587664), (7, 1.9318592e-05), (8, 0.030846793), (9, 3.341703e-05)]
[(0, 3.352789e-05), (1, 4.435749e-05), (2, 0.91685396), (3, 6.491099e-05), (4, 3.603173e-05), (5, 4.8061953e-05), (6, 0.08281749), (7, 2.5629146e-05), (8, 3.171497e-05), (9, 4.433294e-05)]
[(0, 3.4160592e-05), (1, 0.27529243), (2, 4.8180496e-05), (3, 0.7217641), (4, 0.002674906), (5, 4.8968934e-05), (6, 3.372441e-05), (7, 2.6112793e-05), (8, 3.2313463e-05), (9, 4.5169545e-05)]
[(0, 4.0220806e-05), (1, 5.3212236e-05), (2, 0.39102766), (3, 7.786857e-05), (4, 4.322447e-05), (5, 0.5229898), (6, 3.9707244e-05), (7, 3.0745297e-05), (8, 0.0856444), (9, 5.3182786e-05)]
[(0, 2.8163071e-05), (1, 0.1647215), (2, 0.7715104), (3, 5.4524357e-05), (4, 3.0266272e-05), (5, 4.037153e-05), (6, 2.7803468e-05), (7, 0.06352306), (8, 2.664024e-05), (9, 3.72392e-05)]
[(0, 2.930113e-05), (1, 0.23116009), (2, 0.7685626), (3

[(0, 4.022443e-05), (1, 5.321703e-05), (2, 5.6733003e-05), (3, 0.024067827), (4, 4.3228367e-05), (5, 0.9755771), (6, 3.971082e-05), (7, 3.074807e-05), (8, 3.8049417e-05), (9, 5.3187578e-05)]
[(0, 2.2249505e-05), (1, 2.9436158e-05), (2, 3.138096e-05), (3, 4.3075546e-05), (4, 2.3911085e-05), (5, 0.9997605), (6, 2.196541e-05), (7, 1.7007806e-05), (8, 2.1046431e-05), (9, 2.9419865e-05)]
[(0, 2.7737786e-05), (1, 3.669717e-05), (2, 3.9121696e-05), (3, 5.370114e-05), (4, 0.16325316), (5, 0.44504008), (6, 0.029936476), (7, 2.1203117e-05), (8, 0.012941022), (9, 0.34865075)]
[(0, 3.166387e-05), (1, 4.1891388e-05), (2, 0.10367628), (3, 0.30392882), (4, 0.037907943), (5, 0.053243723), (6, 0.1707731), (7, 0.093718015), (8, 2.9951743e-05), (9, 0.2366486)]
[(0, 0.015890483), (1, 3.414074e-05), (2, 3.6396366e-05), (3, 4.9960123e-05), (4, 0.090489835), (5, 0.19673137), (6, 0.044280455), (7, 1.9726049e-05), (8, 2.4410141e-05), (9, 0.65244323)]
[(0, 2.5425514e-05), (1, 3.3638025e-05), (2, 3.5860437e-05),